## 7.2 アニメーショングラフ

In [1]:
import json
import pandas as pd
from sklearn.datasets import load_diabetes

from plotly import graph_objects as go
from plotly import express as px
from plotly.graph_objs.layout import Template

# DiabetesデータセットのDataFrameを読み込み
df_X, df_y = load_diabetes(return_X_y=True, as_frame=True, scaled=False)
df = pd.concat([df_X, df_y], axis=1)

df

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,59.0,2.0,32.1,101.00,157.0,93.2,38.0,4.00,4.8598,87.0,151.0
1,48.0,1.0,21.6,87.00,183.0,103.2,70.0,3.00,3.8918,69.0,75.0
2,72.0,2.0,30.5,93.00,156.0,93.6,41.0,4.00,4.6728,85.0,141.0
3,24.0,1.0,25.3,84.00,198.0,131.4,40.0,5.00,4.8903,89.0,206.0
4,50.0,1.0,23.0,101.00,192.0,125.4,52.0,4.00,4.2905,80.0,135.0
...,...,...,...,...,...,...,...,...,...,...,...
437,60.0,2.0,28.2,112.00,185.0,113.8,42.0,4.00,4.9836,93.0,178.0
438,47.0,2.0,24.9,75.00,225.0,166.0,42.0,5.00,4.4427,102.0,104.0
439,60.0,2.0,24.9,99.67,162.0,106.6,43.0,3.77,4.1271,95.0,132.0
440,36.0,1.0,30.0,95.00,201.0,125.2,42.0,4.79,5.1299,85.0,220.0


In [2]:
def age_to_gen(age:int) -> str:
    """年齢を世代に変換する

    Args:
        age (int): 年齢

    Returns:
        str: 世代を表す文字列
    """
    if age < 20:
        gen = 'under20' # 20歳未満
    elif age < 40:
        gen = 'under40' # 20歳以上40歳未満
    elif age < 60:
        gen = 'under60' # 40歳以上60歳未満
    else:
        gen = 'over60'  # 60歳以上
    return gen

# 列「age」から変換した年代で列「generation」を追加
df['generation'] = df['age'].map(age_to_gen)
df = df.sort_values('age')  # 列「age」で昇順ソート

df

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target,generation
26,19.0,1.0,19.2,87.00,124.0,54.0,57.0,2.00,4.1744,90.0,137.0,under20
374,19.0,1.0,23.2,75.00,143.0,70.4,52.0,3.00,4.6347,72.0,140.0,under20
344,19.0,1.0,25.3,83.00,225.0,156.6,46.0,5.00,4.7185,84.0,200.0,under20
79,20.0,1.0,22.9,87.00,191.0,128.2,53.0,4.00,3.8918,85.0,113.0,under40
226,20.0,2.0,22.1,87.00,171.0,99.6,58.0,3.00,4.2047,78.0,77.0,under40
...,...,...,...,...,...,...,...,...,...,...,...,...
211,74.0,1.0,29.8,101.00,171.0,104.8,50.0,3.00,4.3944,86.0,70.0,over60
311,75.0,1.0,30.1,78.00,222.0,154.2,44.0,5.05,4.7791,97.0,180.0,over60
321,75.0,1.0,31.2,117.67,229.0,138.8,29.0,7.90,5.7236,106.0,230.0,over60
402,79.0,2.0,23.3,88.00,186.0,128.4,33.0,6.00,4.8122,102.0,168.0,over60


In [3]:
# Plotly ExpressからヒストグラムのFigureを作成
figure = px.histogram(
    df,
    x='bmi',                        # 分布を表示する変数
    animation_frame='generation'    # アニメーションに使用する変数
)   # BMIのヒストグラム

# Traceの更新
figure.update_traces({
    'xbins': {
        'start': 10.,   # スタート位置10
        'size': 2.      # ビン幅2
    }   # ヒストグラムのビン
})

# 独自テンプレートの読み込み
with open('custom_white.json') as f:
    custom_white_dict = json.load(f)
    template = Template(custom_white_dict)

# Layoutの作成と更新
layout = go.Layout(
    template=template,
    title='Diabetes dataset',   # グラフタイトル
    xaxis={
        'range': [10, 50]       # x軸範囲
    },
    yaxis={
        'range': [0, 50]        # y軸範囲
    }
)
figure.update_layout(layout)

figure

In [4]:
# スライダー左のボタンを消去
figure['layout'].pop('updatemenus')

figure

In [5]:
generations = {
    'under20': '~ 19 years old',
    'under40': '20 ~ 39 years old',
    'under60': '40 ~ 59 years old',
    'over60': '60 years old ~'
}

# Frameのlistを作成
frames = []
for key, value in generations.items():
    # Frameに追加するTraceの作成
    trace = go.Histogram(
        x=df[df['generation'] == key]['bmi'],
        xbins={
            'start': 10.,
            'size': 2.
        }
    )   # 世代別のBMIのヒストグラム
    
    # Frameの作成
    frame = go.Frame(
        data=trace, 
        name=key, 
        layout={'title': value}
    )

    frames.append(frame)    # Frameを追加 

frames

[Frame({
     'data': [{'type': 'histogram', 'x': array([19.2, 23.2, 25.3]), 'xbins': {'size': 2.0, 'start': 10.0}}],
     'layout': {'title': {'text': '~ 19 years old'}},
     'name': 'under20'
 }),
 Frame({
     'data': [{'type': 'histogram',
               'x': array([22.9, 22.1, 24.2, 20.1, 24.2, 18.6, 23. , 19.9, 19.3, 22.6, 18. , 18.8,
                           29. , 20.7, 25.3, 22.5, 22.6, 23.5, 26. , 24.3, 28.3, 18.8, 30.3, 19.6,
                           24.8, 22.6, 33.6, 25.4, 19.2, 24.7, 22.1, 30.4, 25.5, 31.5, 24.2, 35. ,
                           19.4, 30. , 18.1, 21. , 26. , 20.9, 21.3, 25.2, 19. , 23.3, 29.7, 35.3,
                           25.4, 26.5, 31. , 22. , 31.4, 30.5, 27.8, 18.9, 25.3, 25.4, 19.5, 20.8,
                           35. , 20.6, 24.7, 35.5, 22.6, 20.6, 30. , 25.3, 31.4, 26.3, 21.2, 33. ,
                           29.2, 25.5, 25.2, 25.9, 20.4, 41.3, 28.7, 26.8, 21.1, 24.1, 27.8, 39.1,
                           30. , 22.8, 21.9, 32.3, 22. , 24.1,

In [6]:
# 再生ボタン
play_button = {
    'args': [
        None,
        {'fromcurrent': True}    # 現在位置から再生再開する
    ],
    'label': 'Play',
    'method': 'animate'
}

# 一時停止ボタン
pause_button = {
    'args': [
        [None],
        {'mode': 'immediate'}   # 停止するために必要
    ],
    'label': 'Pause',
    'method': 'animate'
}

# ボタンメニュー
button_menu = {
    'buttons': [play_button, pause_button],
    'direction': 'left',    # 2つのボタンを並べる方向
    'xanchor': 'left',      # xアンカー位置（左）
    'yanchor': 'top',       # yアンカー位置（上）
    'x': -0.1,              # x位置
    'y': -0.15,             # y位置
    'type': 'buttons'
}

# Layoutの作成
layout = go.Layout(
    template=template,
    xaxis={
        'title': 'BMI',
        'range': [10, 50]
    },
    yaxis={
        'title': 'count',
        'range': [0, 50]
    },                    
    updatemenus=[button_menu],
)

# Figureの作成
figure = go.Figure(
    data=frames[0]['data'],     # 最初に表示するグラフ
    frames=frames,
    layout=layout
)

figure

In [7]:
# スライダーに表示するラベル
labels = ['<20', '<40', '<60', '>=60']

# ステップを作成
steps = []
for generation, label in zip(generations.keys(), labels):    
    steps.append({
        'args': [
            [generation],
        ],
        'label': label,
        'method': 'animate'
    })

# スライダーを作成
sliders = [{
    'len': 0.95,    # スライダー長さ
    'x': 0.05,      # スライダー左位置
    'steps': steps  # 
}]

# Layoutを更新
layout.update(
    sliders=sliders
)

# Figureを作成
figure = go.Figure(
    data=frames[0]['data'],     # 最初に表示するグラフ
    frames=frames,
    layout=layout
)

figure

In [8]:
# ドロップダウンの作成
dropdowns = []
for generation, label in zip(generations.keys(), labels):
    dropdowns.append({
        'args': [[generation]],
        'label': label,
        'method': 'animate',
    })

# ドロップダウンメニューの作成
dropdown_menu = {
    'type': 'dropdown',
    'buttons': dropdowns,
    'x': 1.0,   # x位置
    'y': 1.2    # y位置
}

# Layoutの作成
layout = go.Layout(
    template=template,
    xaxis={
        'title': 'BMI',
        'range': [10, 50]
    },
    yaxis={
        'title': 'count',
        'range': [0, 50]
    },   
    updatemenus=[dropdown_menu],
)

# Figureの作成
figure = go.Figure(data=frames[0]['data'], layout=layout, frames=frames)

figure